## 나이브 베이즈 분류

베이즈 정리에 기초한 확률적 분류법 으로써, 오늘날 머신러닝 에서 지도학습 의 방법으로 다방면의 분야에서 사용되고 있다.

나이브 베이즈, 즉, 베이즈 정리를 이용한 알고리즘은 기존에 확보한 데이터가 어떤 확률로 어떻게 분류되었는지 먼저 학습시키고, 앞서 학습된 확률로 구성된 분류 기준들을 이용하여 확률적으로 더 높은 범주에 새로운 데이터를 분류할 수 있게 해준다.

<img src="note/001.png">
<img src="note/002.png">
<img src="note/003.png">
<img src="note/004.png">
<img src="note/005.png">
<img src="note/006.png">
<img src="note/007.png">
<img src="note/008.png">
<img src="note/009.png">
<img src="note/010.png">
<img src="note/011.png">
<img src="note/012.png">
<img src="note/013.png">
<img src="note/014.png">
<img src="note/015.png">
<img src="note/016.png">

각 특성을 개별로 취급해 파라미터를 학습하고 각 특성에서 클래스별 통계를 단순하게 취합하기 때문에 효과적입니다.

scikit-learn 에 구현된 나이브 베이즈 분류기는

- GaussianNB
    - 연속적인 데이터를 위한
- BernoulliNB
    - 이진 데이터를 위한
- MultinomialNB
    - 카운트 데이터를 위한
    - 특성이 어떤 것을 헤아린 정수 카운트
        - 문장에 들어 있는 특정 단어의 갯수들

In [48]:
import numpy as np

X = np.array([[0, 1, 0, 1],
             [1, 0, 1, 1],
             [0, 0, 0, 1],
             [1, 0, 1, 0]])

# 클래스 (이렇게 이유는 찾게되면. 유의미한 의미가 있다면 발표자료 업로드!!!) (Label)
y = np.array([0, 1, 0, 1])

In [50]:
counts = {}
for label in np.unique(y):
    # 각 클래스의 
    counts[label] = X[y == label].sum(axis=1)
    
print("특성 카운트:\n{}".format(counts))

특성 카운트:
{0: array([2, 1]), 1: array([3, 2])}


## Gaussian Bayes

In [51]:
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB  # Gaussian naive Bayes classifier
from sklearn.preprocessing import LabelEncoder
from sklearn.datasets import load_iris

iris = pd.read_csv("./note/Iris.csv")
print(iris.shape)
iris.head(3)

(150, 6)


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa


In [52]:
le = LabelEncoder()
le.fit(iris['Species'])
iris['Species'] = le.transform(iris['Species'])

# Split the dataset into 2/3 training data and 1/3 test data
trainSet, testSet = train_test_split(iris, test_size = 0.33)
print(trainSet.shape)
print(testSet.shape)
trainSet.head(3)

(100, 6)
(50, 6)


,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
1,2,4.9,3.0,1.4,0.2,0
147,148,6.5,3.0,5.2,2.0,2
105,106,7.6,3.0,6.6,2.1,2


In [53]:
# Format the data and expected values for SKLearn
trainData = pd.DataFrame.as_matrix(trainSet[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']])
trainTarget = pd.DataFrame.as_matrix(trainSet[['Species']]).ravel()
testData = pd.DataFrame.as_matrix(testSet[['SepalLengthCm', 'SepalWidthCm', 'PetalLengthCm', 'PetalWidthCm']])
testTarget = pd.DataFrame.as_matrix(testSet[['Species']]).ravel()

classifier = GaussianNB()
classifier.fit(trainData, trainTarget)

GaussianNB(priors=None)

In [56]:
trainData.ndim

2

In [21]:
predictedValues = classifier.predict(testData)

nErrors = (testTarget != predictedValues).sum()
accuracy = 1.0 - nErrors / testTarget.shape[0]
print("Accuracy: ", accuracy)

Accuracy:  0.96


0.95999999999999996

## Multinomial Bayes

In [34]:
from sklearn.datasets import fetch_20newsgroups

news = fetch_20newsgroups(subset="all")
X = news.data
y = news.target

In [46]:
y

array([10,  3, 17, ...,  3,  1,  7])

In [42]:
from sklearn.feature_extraction.text import TfidfVectorizer, HashingVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model1 = Pipeline([
            ('vect', CountVectorizer()), 
            ('clf', MultinomialNB()),
         ])

In [43]:
%%time
from sklearn.model_selection import cross_val_score, KFold

for i, clf in enumerate([model1]):
    scores = cross_val_score(clf, X, y, cv=5)
    print(("Model{0:d}: Mean score: {1:.3f})").format(i, np.mean(scores)))

Model0: Mean score: 0.855)
CPU times: user 28.7 s, sys: 823 ms, total: 29.5 s
Wall time: 29.6 s
